In [1]:
import scipy.integrate as integrate 
import math
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.integrate as integrate 
import cmath 
from scipy import optimize
from scipy.misc import derivative

In [2]:
# Begin with defining everything
# These are the coupling constants of the lagrangian, and the masses of the particles

eps = 0.0000001

#yukawa coupling constant of fermion f 
#(for the top quark)
y_f = 0.99                                          


#EW - sector coupling constants g (SU(2)_L) and g_p (U(1)_Y) 
g = 0.65
g_p = 0.36

# Effective mass of fermion f acquired through yukawa coupling w/ the Higgs
def m_f(phi):
    return y_f**2/2*phi**2 + eps

# W and Z boson masses
def m_W(phi):
    return (g**2/4)*phi**2 + eps

def m_Z(phi):
    return (g**2 + g_p**2)*phi**2/4 + eps

# Higgs and Goldstone masses
def m_h(phi):                                               
    return -3*lam*phi**2 + mu**2 + (15/4)*phi**4/Lam**2 + eps

    #m_h 0 at h_min = (2/5)*(lam/c6)*(1 - np.sqrt(1-5/3*c6*(mu**2)/lam**2))

def m_g(phi):
    return -lam*phi**2 + mu**2 + (3/4)*phi**4/Lam**2 + eps


In [3]:
# different parts of the 1-loop effective potential calculated in the ms-scheme.
# separating the contribution coming from gauge bosons and other particles in the loop correction. 
# obs! input for the first order corrections are the squared masses, i.e. m**2 and not m.  

def v_tree(phi):
    return mu**2/2*phi**2 - lam/4*phi**4 + 1/(8*Lam**2)*phi**6

def v_loop_g(m):
    #Coleman-Weinberg potential for gauge bosons as a function of mass squared.
    scale = 91
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 5/6)

def v_loop(m):
    #Coleman-Weinberg potential for particles as a function of mass squared.
    scale = 91
    return m**2/(64*np.pi**2)*(cmath.log(m/scale) - 3/2)


def v_1(phi):
    return np.real(v_loop(m_h(phi)) + 3*v_loop(m_g(phi)) - 12*v_loop(m_f(phi))
             + 3*v_loop_g(m_Z(phi)) + 6*v_loop_g(m_W(phi)))

#this is the temperature independent part of the effective potential
def v_eff_0(phi):
    return np.real(v_tree(phi) + v_1(phi))


#defining high T expansion of the bosonic (j_b) and fermionic (j_f) thermal functions
def j_b(y):
    a_b = np.pi**2*np.exp(3/2 - 2*0.5572)
    return (np.pi**2/12)*y**2 - np.pi/6*y**3 - (1/32)*y**4*cmath.log(y**2/a_b)

def j_f(y):
    a_f = 16*np.pi**2*np.exp(3/2 - 2*0.5572)
    return -(np.pi**2/24)*y**2 - (1/32)*y**4*cmath.log(y**2/a_f)

def v_eff_T(phi,T):
    return T**4/(2*np.pi**2)*(j_b(cmath.sqrt(m_h(phi))/T) + 3*j_b(cmath.sqrt(m_g(phi))/T) + 3*j_b(cmath.sqrt(m_Z(phi))/T)
                             + 6*j_b(cmath.sqrt(m_W(phi))/T) - 12*j_f(cmath.sqrt(m_f(phi))/T))


In [4]:
# Loading the parameter values to be used in the GW spectrum calculations
# the parameters are mu, lambda and c6 which go into the tree-level effective potential. They have been chosen
# to give the correct higgs mass (125) and vaccum (246) for the 1-loop effective potential at zero temperature.

# The parameters are saved in a list, and need to be reshaped such that each row is giving a set of 
# mu, lambda, c6 that together satisfy the conditions mentioned above.

parameters = np.loadtxt('param34.txt').reshape(-1,3)
#print(parameters)

In [5]:
def find_t_crit(t_span, tolerance):
    T = np.mean(t_span)
    
    def v_eff_T(phi):
        return v_eff(phi,T)
    
    symm_vac = optimize.fmin(v_eff_T, 0, disp = 0)[0]
    broken_vac = optimize.fmin(v_eff_T, 300, disp = 0)[0]
    
    
    min1 = v_eff_T(symm_vac)
    min2 = v_eff_T(broken_vac)
    
    if np.abs(symm_vac - broken_vac) <= tolerance:
        #this is condition for T being too high and only in symmetric phase, Tis to high.
        return find_t_crit([t_span[0], np.mean(t_span)], tolerance)
    
    elif np.abs(min2 - min1) <= tolerance:
        #condition for being at the critical temperature
        return T, broken_vac
    
    elif min2 - min1 > tolerance:
        #the broken phase is higher up than the symmetric one, Temperature is to high
        return find_t_crit([t_span[0], np.mean(t_span)],tolerance)
    else:
        return find_t_crit([np.mean(t_span), t_span[1]],tolerance) 
    

In [6]:
# Overshoot-undershoot implementation to find the bounce solution. Works in the same way as finding the critical temp
# algorithm get's stuck somtetimes due to np.mean not being sensitive enough(?).

def over_undershoot(init_span, tol, max_i, iter = None):
    
    if iter is None:
        iter = 0
        
    iter +=1    
    
    if iter == max_i:
        
        print('arrived at max iterations')
        print('final span: ', init_span)
        
    else:
        t_span = np.array([eps, 1])
        times  = np.linspace(t_span[0], t_span[1], 100) 
        
        
        init = np.mean(init_span)
        
        
        y0 = np.array([init, 0])
    
        soln = solve_ivp(f, t_span, y0, t_eval = times, method = 'Radau')
        #print(soln.message)   
    
    
        t = soln.t
        phi_b = soln.y[0] 
    
    
        if np.abs(np.min(phi_b) - symm_vac) <= tol:
            # Here we have the bounce solution. r_max is the bubble radius, smallest value of the solution which satisfies
            # the bounce condition and run the solution again for that 
            index_r = np.where(np.abs(phi_b - symm_vac) <= tol)
            r_max = t[np.min(index_r)]
            
            t_span = np.array([eps, r_max])
            times  = np.linspace(t_span[0], t_span[1], 100) 
            y0 = np.array([init, 0])
    
            soln = solve_ivp(f, t_span, y0, t_eval = times, method = 'Radau')
            
            r = soln.t
            phi_b = soln.y[0] 
            
            #print('Found it! y0=', init)
            return phi_b, t, soln.y[1], init, r_max
    
        elif phi_b[-1] < symm_vac - tol or phi_b[-1] > broken_vac:
            #condition for rolling down to the left towards phi = -inf &
            #condition for rolling down to the right towards phi = +inf  
            #This means we are too far high up in the span and initial condition is to high.
        
            #print('iteration: ', iter)
            #print('y0 = ', init)
            #print('overshoot')
            #print('final value of the solution:', phi_b[-1])
            #print('\n')
            return over_undershoot([init_span[0], init], tol, max_i, iter)
        
        else:  #phi_b[-1] > symm_vac + tol and phi_b[-1] < init:
            # All other scenarios are undershoot.
            # The initial value is to low need to look higher up in the interval for the correct initial value
            
            #print('iteration: ', iter)
            #print('y0 = ',  init)
            #print('undershoot')
            #print('smallest value of the solution:' , np.min(phi_b))
            #print('And it ended at', phi_b[-1])
            #print('\n')
            return over_undershoot([init, init_span[1]],tol, max_i, iter)

In [7]:
# Defining different functions needed for later use


def v_eff(phi,T):
        return np.real(v_eff_0(phi) + v_eff_T(phi,T)) - np.real(v_eff_0(0) + v_eff_T(0,T)) 
    
def v_eff_tn(phi):
                return np.vectorize(v_eff)(phi,T)


#derivative of v_eff at temperature T
def dv_eff(phi):
    return derivative(v_eff_tn, phi, dx = 0.00001, order=9)    


#Defining system of ODE's to be solved to get the bounce action
def f(t,y):    
    A = y[0]
    B = y[1]
    
    dA_dt = B
    dB_dt = dv_eff(A) - (2/t)*B                                
    return np.array([dA_dt, dB_dt])

def dv_dT(phi,T):
    # Fourth order finite difference scheme for approximation of derivative off effective action along T
    h = 1.4901161193847656e-8
    
    return np.real(-v_eff(phi, T + 2*h) + 8*v_eff(phi, T + h) - 8*v_eff(phi,T - h)
                    + v_eff(phi, T - 2*h))/(12*h)


def d_p1(T, p):
    c2, c1, c0 = p
    return 2*c2*T + c1

In [14]:
# for each set of parameters we now calculate the latent heat, invers nucleation time and nucelation temperature.
GWP = []
for row in parameters:
    s3_T = []
    mu, lam, Lam = row
    
    print('\u03BC, \u03BB, \u039B = ', mu**2, ',', lam,',', Lam)
    
    a = 6*lam + 9/4*g**2 + 3/4*g_p**2 + 3*y_f**2
    
    T_min = 40  #np.real(np.sqrt(12/a)*cmath.sqrt(mu**2))
    
    t_span = [1.01*T_min, 300]
    tolerance = 1
    
    t_c , phi_c = find_t_crit(t_span, tolerance)
    if phi_c/t_c > 1:
        print('critical temperature:', t_c)
        print('strength of phase transition', phi_c/t_c)
        print('\n')
        
        max_iter = 20
        i = 0
    
        #initial guess for span of nucleation temperature and do a binary search in while loop    
        T_span = [0, t_c] 
    
        while i <= max_iter:
            i += 1
            T = (T_span[0] + T_span[1])/2
        
        
            print('i =', i)
            print('T =', T)
            
        
        
            symm_vac = 0

            broken_vac = optimize.fmin(v_eff_tn, 300, disp = 0)[0]
            tol = 2

                
            init_span = np.array([symm_vac, broken_vac])     
                                                      
            phi_b , r , phi_dot, ic, r_max = over_undershoot(init_span, tol, 100)

        
            v_bounce = np.vectorize(v_eff)(phi_b,T)


            #R is the interval over which we want to integrate the functional over to get the bounce action.
            #L is the euclidean lagrangian density to be integrated.  

            R = np.linspace(0, r_max, len(phi_b))
            L = ((1/2)*(phi_dot)**2 + v_bounce)

        
        
            s3 = 4*np.pi*np.trapz((R**2)*L, R)
            
            s3_T.append([T, s3/T])
        
            print('bounce action over temperature', s3/T)
            print('\n')
        
            if 141 >= s3/T >= 139:
                tn = T
                break
            elif s3/T > 141:
                #T is too high need to lower
                T_span = [T_span[0], T]
            else:
                T_span = [T, T_span[1]]
                 
    
        g_star = 106.95
        rho_rad = np.pi**2*g_star*(T**4)/30
        delta_v = v_eff(symm_vac, T) - v_eff(broken_vac,tn)
        delta_dv = dv_eff(symm_vac) - dv_eff(broken_vac)

        alpha = (1/rho_rad)*(delta_v - (1/4)*delta_dv)
    
        dv = np.vectorize(dv_dT)(phi_b,tn)
        
        ds3_dT = 4*np.pi*np.trapz(R**2*dv, R)
        
        p = np.polyfit(np.array(s3_T)[:,0], np.array(s3_T)[:,1], deg = 2)
        
        beta1 = d_p1(tn,p)*tn
        beta = ds3_dT - s3/tn
        
        print('\u03B1, \u03B2, \u03B2 1, T =', alpha, beta, beta1, tn)
        print('\n')
        GWP.append([alpha, beta, beta1, tn, phi_c/t_c])

μ, λ, Λ =  69.12234650009721 , 0.01439504985644152 , 650.8905719777708
critical temperature: 93.03837112188339
strength of phase transition 2.7515490472537283


i = 1
T = 46.51918556094169
bounce action over temperature 121.62619889492562


i = 2
T = 69.77877834141253
bounce action over temperature 229.52093082103403


i = 3
T = 58.14898195117711
bounce action over temperature 150.598431668964


i = 4
T = 52.3340837560594
bounce action over temperature 132.747224870624


i = 5
T = 55.24153285361825
bounce action over temperature 140.6811513825481


α, β, β 1, T = 0.13223321138728056 168.0187419221722 198.08106764613314 55.24153285361825


μ, λ, Λ =  73.84376303162382 , 0.014162076668998208 , 653.5382287633015
critical temperature: 93.25833855569363
strength of phase transition 2.7546613587603797


i = 1
T = 46.62916927784681
bounce action over temperature 122.44091160381315


i = 2
T = 69.94375391677022
bounce action over temperature 230.7706289961499


i = 3
T = 58.28646159730852
boun

critical temperature: 99.18910945355891
strength of phase transition 2.7882258769810155


i = 1
T = 49.59455472677946
bounce action over temperature 119.23771022428157


i = 2
T = 74.39183209016919
bounce action over temperature 231.66391956937883


i = 3
T = 61.993193408474326
bounce action over temperature 150.41719573647626


i = 4
T = 55.79387406762689
bounce action over temperature 131.6153270539215


i = 5
T = 58.89353373805061
bounce action over temperature 140.02230184587773


α, β, β 1, T = 0.13380945116214485 177.74494927947177 208.7422539513676 58.89353373805061


μ, λ, Λ =  11.112941783001515 , 0.006127504591208684 , 724.5854806239367
critical temperature: 100.90013683438303
strength of phase transition 2.7696326193384153


i = 1
T = 50.45006841719152
bounce action over temperature 117.94799938372906


i = 2
T = 75.67510262578728
bounce action over temperature 227.87480264808008


i = 3
T = 63.062585521489396
bounce action over temperature 148.27278491730405


i = 4
T = 56.

In [15]:
print(GWP)

[[0.13223321138728056, 168.0187419221722, 198.08106764613314, 55.24153285361825, 2.7515490472537283], [0.14055782557358845, 156.54703750693747, 183.13675459066363, 54.643557747476734, 2.7546613587603797], [0.11757420039050158, 195.74299682913085, 233.7293559030371, 56.42762233112008, 2.7567789777096245], [0.11758124887659101, 195.6584058991339, 233.3200565434204, 57.064680595463145, 2.75659756670854], [0.1337741386843011, 177.79311917676893, 209.5256344053903, 54.751379479561, 2.7931122983355476], [0.1251319671501426, 184.27272955611227, 218.42678651543469, 57.31298216212307, 2.7637923481412874], [0.1254578456956374, 186.1609115957197, 221.00064208981632, 57.693813645653435, 2.773024675529104], [0.1254367537190669, 186.07860827815688, 220.89616867417024, 58.00510359235342, 2.7720392076283376], [0.13368923645115144, 176.59058903341557, 207.64975203679813, 58.65723035875709, 2.784821826965446], [0.15034105103603404, 150.43375379587226, 171.3193347633817, 58.26450295718388, 2.776719332506

In [16]:
GWP_file = open('GWparam34.txt', 'w')

for row in GWP:
    np.savetxt(GWP_file, row)
    
GWP_file.close()

In [17]:
og_array = np.loadtxt('GWparam34.txt').reshape(-1,5)


print('T= ', np.mean(og_array[:,3]), '\n') 
for row in og_array:
    print(row[0],',', row[2])  

T=  58.01536042339595 

0.13223321138728056 , 198.08106764613314
0.14055782557358845 , 183.13675459066363
0.11757420039050158 , 233.7293559030371
0.11758124887659101 , 233.3200565434204
0.1337741386843011 , 209.5256344053903
0.1251319671501426 , 218.42678651543469
0.1254578456956374 , 221.00064208981632
0.1254367537190669 , 220.89616867417024
0.13368923645115144 , 207.64975203679813
0.15034105103603404 , 171.3193347633817
0.1257055976697485 , 222.77566241394067
0.13380945116214485 , 208.7422539513676
0.12545231134380064 , 220.13614986202924
0.12547882422461762 , 220.741554850174
0.12922175665845545 , 211.1581879332129
